### ITD
En este dataset tenemos fotos tomadas con 10 cámaras diferentes a diferentes telas. De cada cámara tenemos una parte de las fotos para train, en las que solamente hay fotos de telas en buen estado. Y otra parte para test, en las que hay fotos de telas en buen estado y dañadas.
El objetivo es entrenar el modelo solo con las fotos de train (tela en buen estado) y poder predecir si una foto de test es una tela en buen estado ($y_{test}=0$) o dañada ($y_{test}=1$).

## Leer datos todos juntos

Leemos mezclando test y train, y luego particioanremos todos los datos en train, val y test

In [ ]:
import cv2
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

def cargar_dataset_imagenes(root_path, image_size=(64, 64)):
    """
    Lee las imágenes y las devuelve en formato (N, Alto, Ancho).
    NO aplana las imágenes. Mantiene la estructura espacial.
    """
    X_list = []
    y_list = []
    
    root = Path(root_path)
    print(f"📂 Escaneando imágenes en: {root.absolute()} ...")
    
    if not root.exists():
        print("❌ Error: La ruta no existe.")
        return np.array([]), np.array([])

    contador = 0
    
    for img_path in root.rglob('*.*'):
        if img_path.suffix.lower() not in ['.png', '.jpg', '.jpeg', '.bmp']:
            continue
            
        # --- 1. ETIQUETADO ---
        # 1 = Defecto (Anomaly), 0 = Bien (Good)
        label = 1 if 'anomaly' in str(img_path).lower() else 0
            
        # --- 2. LECTURA (Escala de Grises) ---
        # Leemos en blanco y negro para simplificar (1 canal). 
        # Si quisieras color, quita el flag IMREAD_GRAYSCALE.
        img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        
        if img is None: continue
        
        # --- 3. RESIZE (Obligatorio) ---
        # Todas deben medir lo mismo para entrar en el array numpy
        img = cv2.resize(img, image_size)
        
        # --- 4. NORMALIZACIÓN ---
        # Pasamos de 0-255 (enteros) a 0.0-1.0 (float).
        # Esto es vital tanto para ML clásico como para Deep Learning.
        img_norm = img.astype('float32') / 255.0
        
        X_list.append(img_norm)
        y_list.append(label)
        contador += 1

    # Convertimos la lista a un Array Numpy 3D: (N_fotos, Alto, Ancho)
    X_array = np.array(X_list)
    y_array = np.array(y_list)

    print(f"✅ Carga completa. {contador} imágenes procesadas.")
    return X_array, y_array

# --- EJECUCIÓN: CARGAR Y DIVIDIR ---

ruta = "./ITD" # Tu ruta
X_total, y_total = cargar_dataset_imagenes(ruta, image_size=(64, 64))

# Verificación de forma
# Debería salir algo como: (5000, 64, 64) -> 5000 fotos de 64x64
print(f"\nForma de X_total: {X_total.shape}") 

# --- PARTICIÓN (Train / Val / Test) ---
# 1. Separamos Train (70%) del resto (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_total, y_total, test_size=0.3, random_state=42, stratify=y_total
)

# 2. Separamos el resto en Val (15%) y Test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("-" * 30)
print(f"🔹 Train shape: {X_train.shape} (Listo para Autoencoders/CNN)")
print(f"🔹 Val shape:   {X_val.shape}")
print(f"🔹 Test shape:  {X_test.shape}")
print("-" * 30)

# mostramos numero de anomalias en cada conjunto
print(f"Train anomalies: {np.sum(y_train)}")
print(f"Val anomalies:   {np.sum(y_val)}")
print(f"Test anomalies:  {np.sum(y_test)}")

📂 Escaneando imágenes en: c:\Users\ander\OneDrive - UPNA\4º\Extracción del conocimiento\TrabajoGrupo\ITD ...
✅ Carga completa. 5878 imágenes procesadas.

Forma de X_total: (5878, 64, 64)
------------------------------
🔹 Train shape: (4114, 64, 64) (Listo para Autoencoders/CNN)
🔹 Val shape:   (882, 64, 64)
🔹 Test shape:  (882, 64, 64)
------------------------------
Train anomalies: 675
Val anomalies:   145
Test anomalies:  145


Ahora la forma de leerlos de train (solo good) y test (good y anomaly) por separado:

In [6]:
import cv2
import numpy as np
from pathlib import Path

def cargar_datos_separados_raw(root_path, image_size=(64, 64)):
    """
    Lee las carpetas originales 'train' y 'test' y devuelve los datos separados.
    Formato: RAW (Píxeles 2D, normalizados 0-1).
    """
    # Listas para Entrenamiento
    X_train = []
    y_train = []
    
    # Listas para Test
    X_test = []
    y_test = []
    
    root = Path(root_path)
    print(f"📂 Escaneando estructura original en: {root.absolute()} ...")
    
    if not root.exists():
        print("❌ Error: La ruta no existe.")
        return np.array([]), np.array([]), np.array([]), np.array([])

    contador_train = 0
    contador_test = 0
    
    for img_path in root.rglob('*.*'):
        if img_path.suffix.lower() not in ['.png', '.jpg', '.jpeg', '.bmp']:
            continue
        
        path_str = str(img_path).lower()
        
        # --- 1. IDENTIFICAR SPLIT (¿Es Train o Test?) ---
        if 'train' in path_str:
            is_train = True
        elif 'test' in path_str:
            is_train = False
        else:
            continue # Si no está en ninguna carpeta train/test, la ignoramos
            
        # --- 2. IDENTIFICAR ETIQUETA (¿Good o Anomaly?) ---
        # 1 = Defecto, 0 = Bien
        label = 1 if 'anomaly' in path_str else 0
        
        # --- 3. LEER RAW (Escala de Grises) ---
        img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        
        if img is None: continue
        
        # --- 4. RESIZE & NORMALIZACIÓN ---
        img = cv2.resize(img, image_size)
        img_norm = img.astype('float32') / 255.0
        
        # --- 5. GUARDAR DONDE TOQUE ---
        if is_train:
            X_train.append(img_norm)
            y_train.append(label)
            contador_train += 1
        else:
            X_test.append(img_norm)
            y_test.append(label)
            contador_test += 1

    # Convertir todo a Numpy Arrays
    X_train_arr = np.array(X_train)
    y_train_arr = np.array(y_train)
    X_test_arr = np.array(X_test)
    y_test_arr = np.array(y_test)

    print(f"✅ Carga finalizada.")
    print(f"   Train: {contador_train} imágenes.")
    print(f"   Test:  {contador_test} imágenes.")
    
    return X_train_arr, y_train_arr, X_test_arr, y_test_arr

# --- USO ---
ruta = "./ITD" # Asegúrate que es la ruta correcta
X_train, y_train, X_test, y_test = cargar_datos_separados_raw(ruta, image_size=(64, 64))

print("\n--- DIMENSIONES (RAW) ---")
# Debería salir (N, 64, 64) -> Datos 2D listos para lo que quieras
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test:  {X_test.shape}")
print(f"y_test:  {y_test.shape}")

📂 Escaneando estructura original en: c:\Users\Mikel\Desktop\EXTRACCIÓN DEL CONOCIMIENTO\Trabajo\Extraccion\ITD ...
✅ Carga finalizada.
   Train: 4391 imágenes.
   Test:  1141 imágenes.

--- DIMENSIONES (RAW) ---
X_train: (4391, 64, 64)
y_train: (4391,)
X_test:  (1141, 64, 64)
y_test:  (1141,)


In [2]:
# Ejemplo rápido para Isolation Forest
# -1 le dice a numpy: "calcula tú esta dimensión" (que será 64*64 = 4096)
X_train_flat = X_train.reshape(X_train.shape[0], -1) 
X_test_flat = X_test.reshape(X_test.shape[0], -1)

print(f"Forma aplanada para ML Clásico: {X_train_flat.shape}") 
# Resultado: (N, 4096)

# clf.fit(X_train_flat) ...

Forma aplanada para ML Clásico: (4391, 4096)


In [ ]:
# Ejemplo mental para futuro (PyTorch)
# Solo tendrías que añadir una dimensión de canal: (N, 1, 64, 64)
# tensor_img = torch.from_numpy(X_train).unsqueeze(1)

Entrenar y predecir con OneClass + SVM. 
El One-Class SVM no es un clasificador supervisado tradicional (como un SVM binario con clases 0 y 1).
Es un método de detección de anomalías o novelty detection, cuyo objetivo es aprender la forma del conjunto de datos “normales” y luego identificar puntos que no se ajustan a esa distribución. 

In [8]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

X_train_good = X_train_flat[y_train == 0]

print(f"Entrenando One-Class SVM con {X_train_good.shape[0]} imágenes buenas...")

ocsvm = OneClassSVM(
    kernel="rbf",     # kernel radial: suele funcionar bien para este tipo de datos
    nu=0.01,          # proporción esperada de anomalías
    gamma="scale"     # parámetro del kernel RBF (puede ajustarse)
)

ocsvm.fit(X_train_good)
print("✅ Entrenamiento completado.")


y_pred_test = ocsvm.predict(X_test_flat)

# Convertimos a 0 = normal, 1 = anómalo
y_pred_test = np.where(y_pred_test == -1, 1, 0)



Entrenando One-Class SVM con 4391 imágenes buenas...
✅ Entrenamiento completado.


In [9]:
# ------------------------------------------------------------
# 4️⃣ Evaluación
# ------------------------------------------------------------
print("\n📊 MATRIZ DE CONFUSIÓN (Test):")
print(confusion_matrix(y_test, y_pred_test))

print("\n📈 MÉTRICAS (Test):")
print(classification_report(y_test, y_pred_test, digits=3))

try:
    auc = roc_auc_score(y_test, y_pred_test)
    print(f"ROC-AUC (Test): {auc:.3f}")
except Exception as e:
    print("No se pudo calcular AUC:", e)

detected_anomalies = np.sum(y_pred_test)
print(f"\n🔍 Anomalías detectadas en test: {detected_anomalies} / {len(y_test)}")


📊 MATRIZ DE CONFUSIÓN (Test):
[[500  12]
 [431 198]]

📈 MÉTRICAS (Test):
              precision    recall  f1-score   support

           0      0.537     0.977     0.693       512
           1      0.943     0.315     0.472       629

    accuracy                          0.612      1141
   macro avg      0.740     0.646     0.582      1141
weighted avg      0.761     0.612     0.571      1141

ROC-AUC (Test): 0.646

🔍 Anomalías detectadas en test: 210 / 1141


## Autoencoder

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import roc_auc_score, classification_report

# ------------------------------------------------------------
# 1️⃣ Preprocesamiento
# ------------------------------------------------------------
# Si tus datos son (N, 64, 64) -> agrega canal (N, 64, 64, 1)
def preparar_datos(X):
    X = np.expand_dims(X, axis=-1)
    return X.astype('float32')

X_train_prep = preparar_datos(X_train)
X_test_prep  = preparar_datos(X_test)

# ------------------------------------------------------------
# 2️⃣ Definición del modelo Autoencoder (CNN)
# ------------------------------------------------------------
def build_autoencoder(input_shape=(64, 64, 1)):
    encoder = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2))
    ])
    
    decoder = models.Sequential([
        layers.Conv2DTranspose(64, (3,3), strides=2, activation='relu', padding='same'),
        layers.Conv2DTranspose(32, (3,3), strides=2, activation='relu', padding='same'),
        layers.Conv2D(1, (3,3), activation='sigmoid', padding='same')
    ])
    
    inp = layers.Input(shape=input_shape)
    encoded = encoder(inp)
    decoded = decoder(encoded)
    
    autoencoder = models.Model(inp, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    
    return autoencoder

autoencoder = build_autoencoder(input_shape=(64, 64, 1))
autoencoder.summary()

# ------------------------------------------------------------
# 3️⃣ Entrenamiento
# ------------------------------------------------------------
# Si usas el esquema One-Class:
# solo imágenes buenas (sin anomalías)
X_train_good = X_train_prep[y_train == 0] if 'y_train' in locals() else X_train_prep

history = autoencoder.fit(
    X_train_good, X_train_good,
    epochs=50,
    batch_size=32,
    shuffle=True,
    validation_split=0.1,  # o usa X_val si tienes el split separado
    verbose=1
)

# ------------------------------------------------------------
# 4️⃣ Evaluación (detección de anomalías)
# ------------------------------------------------------------
# 1. Reconstruimos las imágenes del test
X_test_pred = autoencoder.predict(X_test_prep)

# 2. Calculamos el error de reconstrucción
recon_error = np.mean(np.square(X_test_prep - X_test_pred), axis=(1,2,3))

# 3. Fijamos un umbral (por ejemplo, percentil 95 de los errores de train)
threshold = np.percentile(
    np.mean(np.square(X_train_good - autoencoder.predict(X_train_good)), axis=(1,2,3)),
    95
)

# 4. Clasificamos según el umbral
y_pred = (recon_error > threshold).astype(int)

print("\n📊 RESULTADOS (Test):")
print(classification_report(y_test, y_pred, digits=3))
print(f"ROC-AUC: {roc_auc_score(y_test, recon_error):.3f}")
print(f"Umbral elegido: {threshold:.6f}")




ModuleNotFoundError: No module named 'tensorflow'